In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pickle

Two cases, each one associated with a gold synthesis, gold keyword sequence and the matter

In [2]:
gold_synth1 = "il résulte de la combinaison des articles 455 et 783, alinéa 2, du code de procédure civile, dans sa version antérieure au décret n° 2019-1333 du 11 décembre 2019, que sont recevables les conclusions postérieures à l'ordonnance de clôture aux termes desquelles une partie en demande la révocation et qu'il appartient au juge qui en est saisi d'y répondre"
gold_synth2 = "il résulte de la combinaison des articles l. 7313-13, alinéa 1, du code du travail et 14 de l'accord national interprofessionnel des voyageurs, représentants, placiers du 3 octobre 1975 que, lorsqu'il est jugé que le licenciement prononcé pour faute grave repose en réalité sur une cause réelle et sérieuse, le bénéfice de l'indemnité spéciale de rupture réclamée par le voyageur représentant placier ne peut être subordonné à la condition de renonciation par le salarié à l'indemnité de clientèle dans le délai de trente jours suivant l'expiration du contrat de travail"
gold_kw1 = "licenciement <t> cause <t> cause réelle et sérieuse <t> cause ne constituant pas une faute grave. <t> voyageur représentant placier (vrp) <t> indemnité spéciale de rupture <t> indemnité de clientèle <t> défaut de renonciation <t> délai de trente jours suivant l'expiration du contrat de travail <t> conditions <t> portée"
gold_kw2 = "procédure de la mise en état <t> ordonnance de clôture <t> dépôt des conclusions des parties <t> dépôt postérieur à l'ordonnance <t> recevabilité <t> cas <t> conclusions demandant la révocation de l'ordonnance de clôture ou le rejet des débats des conclusions ou productions de dernière heure de l'adversaire"
matter1 = "contrat de travail, rupture"
matter2 = "procedure civile"

Load the tokeniser and the model

In [3]:
tokeniser = AutoTokenizer.from_pretrained("rbawden/CCASS-auto-titrages-base", use_auth_token=True)
model = AutoModelForSeq2SeqLM.from_pretrained("rbawden/CCASS-auto-titrages-base", use_auth_token=True)

Predict a keyword sequence from each of the syntheses

In [4]:
inputs = tokeniser([matter1 + " <t> " + gold_synth1], return_tensors='pt')
outputs = model.generate(inputs['input_ids'])
pred_kw1 = tokeniser.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenisation_spaces=True)

inputs = tokeniser([matter2 + " <t> " + gold_synth2], return_tensors='pt')
outputs = model.generate(inputs['input_ids'])
pred_kw2 = tokeniser.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenisation_spaces=True)

print(pred_kw1)
print(pred_kw2)

["licenciement <t> cause <t> cause réelle et sérieuse <t> défaut <t> procédure de révocation <t> conclusions postérieures à la clôture de l'enquête <t> irrecevabilité <t> cas <t> conclusions postérieures à la révocation de l'ordonnance de clôture "]
['sursis à statuer <t> demande <t> rejet <t> portée ']


You can specify to return `n` predicted sequences (by taking the `n`-best in the beam)

In [5]:
n=3
inputs = tokeniser([matter1 + " <t> " + gold_synth1], return_tensors='pt')
outputs = model.generate(inputs['input_ids'], num_beams=n, num_return_sequences=n)
pred_kw1_multiple = tokeniser.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenisation_spaces=True)

inputs = tokeniser([matter2 + " <t> " + gold_synth2], return_tensors='pt')
outputs = model.generate(inputs['input_ids'], num_beams=n, num_return_sequences=n)
pred_kw2_multiple = tokeniser.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenisation_spaces=True)

for pred in pred_kw1_multiple:
    print(pred)
print('---')
for pred in pred_kw2_multiple:
    print(pred)

licenciement <t> cause <t> cause réelle et sérieuse <t> défaut <t> effets <t> indemnité <t> charge <t> détermination 
licenciement <t> cause <t> cause réelle et sérieuse <t> défaut <t> indemnité <t> condition <t> demande postérieures à la clôture de l'instance <t> office du juge 
licenciement <t> cause <t> cause réelle et sérieuse <t> défaut <t> indemnité <t> condition <t> demande postérieures à la clôture de l'instance <t> irrecevabilité <t> cas 
---
sursis à statuer <t> demande <t> rejet <t> portée 
sursis à statuer <t> conditions <t> licenciement pour faute grave <t> défaut <t> applications diverses <t> licenciement <t> indemnité spéciale 
sursis à statuer <t> conditions <t> licenciement pour faute grave <t> défaut <t> applications diverses 


You can now use the predicted keyword sequences (and the gold texts to predict the similarity of the two examples)

In [6]:
from predict_similarity import *

Prepare the inputs so that they are in list format

In [7]:
gold_synth1_list = create_list_from_sommaire(gold_synth1)
gold_synth2_list = create_list_from_sommaire(gold_synth2)
gold_kw1_list = create_list_from_titrage(gold_kw1)
gold_kw2_list = create_list_from_titrage(gold_kw2)
pred_kw1_list = create_list_from_titrage('\t'.join(pred_kw1_multiple))
pred_kw2_list = create_list_from_titrage('\t'.join(pred_kw2_multiple))

print(gold_synth1_list)
print(gold_kw1_list)
print(pred_kw1_list)

[['il', 'résulte', 'de', 'la', 'combinaison', 'des', 'articles', '455', 'et', '783,', 'alinéa', '2,', 'du', 'code', 'de', 'procédure', 'civile,', 'dans', 'sa', 'version', 'antérieure', 'au', 'décret', 'n°', '2019-1333', 'du', '11', 'décembre', '2019,', 'que', 'sont', 'recevables', 'les', 'conclusions', 'postérieures', 'à', "l'ordonnance", 'de', 'clôture', 'aux', 'termes', 'desquelles', 'une', 'partie', 'en', 'demande', 'la', 'révocation', 'et', "qu'il", 'appartient', 'au', 'juge', 'qui', 'en', 'est', 'saisi', "d'y", 'répondre']]
[['licenciement', 'cause', 'cause réelle et sérieuse', 'cause ne constituant pas une faute grave.', 'voyageur représentant placier (vrp)', 'indemnité spéciale de rupture', 'indemnité de clientèle', 'défaut de renonciation', "délai de trente jours suivant l'expiration du contrat de travail", 'conditions', 'portée']]
[['licenciement', 'cause', 'cause réelle et sérieuse', 'défaut', 'effets', 'indemnité', 'charge', 'détermination '], ['licenciement', 'cause', 'caus

Calculate unsupervised similarity features

In [8]:
synth_scores, gold_kw_scores, pred_kw_scores = calculate_sim_features(gold_titrage1=[gold_kw1_list], gold_titrage2=[gold_kw2_list],
                           pred_titrages1=[pred_kw1_list], pred_titrages2=[pred_kw2_list], 
                           sommaires1=[gold_synth1], sommaires2=[gold_synth2])

print(synth_scores)
print(gold_kw_scores)
print(pred_kw_scores)

[1.0]
[0.0]
[0.125]


Load the trained multilayer perceptron model (be careful to get the appropriate one for the input features you have available)

In [9]:
model_path='../models/similarity/edsim.sommaire-gold.titrage-gold.titrage-pred-3.pickle'
similarity_predictions = mlp_predict(model_path, synth_scores, gold_kw_scores, pred_kw_scores)

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


And now you have your predicted similarity score! Ideally they should be between 0 and 3, but there is nothing to constrain the model to do this, so some scores may be higher or lower.

In [10]:
print(similarity_predictions)

[3.84931488]


If you do not have all input texts, there are other models with fewer features (e.g. if there is no gold keyword sequence)

In [11]:
model_path='../models/similarity/edsim.sommaire-gold.titrage-pred-3.pickle'
similarity_predictions = mlp_predict(model_path, synth_scores, None, pred_kw_scores)

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [12]:
print(similarity_predictions)

[3.05540247]
